# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,22.28,72,100,3.87,BR,1730084707
1,1,port-aux-francais,-49.3500,70.2167,2.80,75,98,14.16,TF,1730084986
2,2,pisco,-13.7000,-76.2167,17.23,88,20,4.12,PE,1730084987
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,3.33,45,75,3.00,RU,1730084988
4,4,lebu,-37.6167,-73.6500,9.55,94,1,7.27,CL,1730084855


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='blue',
    alpha=0.6,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Humidity'])

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & 
    (city_data_df['Max Temp'] <= 30) &
    (city_data_df['Humidity'] < 60) &
    (city_data_df['Cloudiness'] < 20)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,tchintabaraden,15.8969,5.7985,25.32,19,2,4.14,NE,1730085000
56,56,margaret river,-33.9500,115.0667,20.94,48,0,2.88,AU,1730085050
132,132,badiyah,22.4500,58.8000,26.02,32,0,1.30,OM,1730085139
133,133,mendoza,-32.8908,-68.8272,22.74,40,0,1.54,AR,1730084342
140,140,yola,9.2000,12.4833,23.56,47,12,0.20,NG,1730085148


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,tchintabaraden,NE,15.8969,5.7985,19,
56,margaret river,AU,-33.9500,115.0667,48,
132,badiyah,OM,22.4500,58.8000,32,
133,mendoza,AR,-32.8908,-68.8272,40,
140,yola,NG,9.2000,12.4833,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "limit": 1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError, requests.RequestException) as e:
        # If no hotel is found or there is an error, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(f"Error finding hotel for {hotel_df.loc[index, 'City']}: {e}")

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
Error finding hotel for tchintabaraden: 401 Client Error: Unauthorized for url: https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&filter=circle%3A5.7985%2C15.8969%2C10000&bias=proximity%3A5.7985%2C15.8969
tchintabaraden - nearest hotel: No hotel found
Error finding hotel for margaret river: 401 Client Error: Unauthorized for url: https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&filter=circle%3A115.0667%2C-33.95%2C10000&bias=proximity%3A115.0667%2C-33.95
margaret river - nearest hotel: No hotel found
Error finding hotel for badiyah: 401 Client Error: Unauthorized for url: https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&filter=circle%3A58.8%2C22.45%2C10000&bias=proximity%3A58.8%2C22.45
badiyah - nearest hotel: No hotel found
Error finding hotel for mendoza: 401 Client Error: Unauthorized for url: https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&filter=circle%3A-68.827

,City,Country,Lat,Lng,Humidity,Hotel Name
13,tchintabaraden,NE,15.8969,5.7985,19,No hotel found
56,margaret river,AU,-33.9500,115.0667,48,No hotel found
132,badiyah,OM,22.4500,58.8000,32,No hotel found
133,mendoza,AR,-32.8908,-68.8272,40,No hotel found
140,yola,NG,9.2000,12.4833,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='blue',
    alpha=0.6,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity'])

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)